In [1]:
import matplotlib.pyplot as plt
import numpy
import numpy as np
import pandas
import pandas as pd
import seaborn as sns
import re

import tensorflow as tf
from pandas import DataFrame

import nbimporter

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)
pd.set_option('display.max_columns', None)

In [2]:
# def merge_biosignals_with_tetris_results(biosignals, tetris_results):
#     tetris_max_time = len(tetris_results)
#     bio_start_time = biosignals.head(1)['Czas'].item()
#     bio_end_time = biosignals.tail(1)['Czas'].item()
#
#     if 0 > (tetris_max_time - (bio_end_time - bio_start_time)) or (tetris_max_time - (bio_end_time - bio_start_time)) > 1:
#         raise ValueError('Tetris times are not compatible with biosignals times')
#
#     biosignals['tetris_result'] = pd.Series([0] * len(biosignals.index), index=biosignals.index)
#     biosignals['tetris_result'] = pd.Series([0] * len(biosignals.index), index=biosignals.index)
#
# def fsdg(tetris_result, biosignals):
#     biosignals['tetris_result'] = biosignals.apply(lambda x: )
#
#
# biosignals = read_biosignal_data('s01/E03_R02_S01.txt')
# range_biosignals = get_biosignal_in_range(biosignals, 0, 62)
# tetris_data = read_tetris_data('s01/tetris_first_round.txt')
# first_player_tetris_result = get_first_player_tetris_state(tetris_data)
#
#
# merge_biosignals_with_tetris_results(range_biosignals, first_player_tetris_result)

In [3]:
def get_biosignal_in_range(biosignals, first_range, second_range):
    return biosignals.loc[(first_range <= biosignals['Czas']) & (biosignals['Czas'] < second_range)]

In [4]:
def read_biosignal_data(biosignal_path):
    biosignals = pd.read_csv(biosignal_path, sep=',', error_bad_lines=False, index_col=False, dtype='unicode').iloc[1:,:11]
    for columnName in biosignals.columns.values:
        biosignals[columnName] = pd.to_numeric(biosignals[columnName], errors='coerce')

    return biosignals

In [5]:
def read_tetris_data(tetris_data_path):
    dataset = pd.read_csv(tetris_data_path, sep='\t', error_bad_lines=False, index_col=False, dtype='unicode')
    dataset['time'] = pd.to_numeric(dataset['time'], errors='coerce')
    dataset['player'] = pd.to_numeric(dataset['player'], errors='coerce')
    
    return dataset

In [6]:
LINES_PATTERN = "(?<=lines\()[^)]*(?=\))"

def get_first_player_tetris_state(data):
    max_time = data.tail(1)['time'].item()
    tetris_result = []
    first_player = 1
    first_player_lines = 0
    second_player_lines = 0
    i = 0

    while i < max_time:
        temp_data = data.loc[data['time']==i]

        if temp_data.empty:
            method_name(first_player_lines,second_player_lines,tetris_result)
            i+=1
            continue

        for index, row in temp_data.iterrows():
            if re.search(LINES_PATTERN, row['move']):
                line_number = int(re.findall(LINES_PATTERN, row['move'])[0])
                if row['player'] == first_player:
                    first_player_lines = line_number
                else:
                    second_player_lines = line_number
                    
        method_name(first_player_lines,second_player_lines,tetris_result)
        i+=1

    return tetris_result

def method_name(first_player_lines, second_player_lines, tetris_result):
    if first_player_lines > second_player_lines:
        tetris_result.append(1)
    elif first_player_lines < second_player_lines:
        tetris_result.append(-1)
    else:
        tetris_result.append(0)


tetris_data = read_tetris_data('s01/tetris_first_round.txt')
first_player_tetris_result = get_first_player_tetris_state(tetris_data)

In [7]:
def second_player_game_state(data):
    first_player = get_first_player_tetris_state(data)

    return [state * -1 for state in first_player]

In [8]:
def merge_biosignals_with_tetris_results(biosignals, tetris_results):
    tetris_max_time = len(tetris_results)
    bio_start_time = biosignals.head(1)['Czas'].item()
    bio_end_time = biosignals.tail(1)['Czas'].item()

    if 0 > (tetris_max_time - (bio_end_time - bio_start_time)) or (tetris_max_time - (bio_end_time - bio_start_time)) > 1:
        raise ValueError('Tetris times are not compatible with biosignals times')

def insert_tetris_result(biosignals, tetris_result):

    biosignals['tetris'] = biosignals.apply(lambda x: tetris_result[int(x['Czas'])], axis=1)
    biosignals['tetris'] = biosignals['tetris'].map({-1: 'Lose', 0: 'Draw', 1: 'Win'})


biosignals = read_biosignal_data('s01/E03_R02_S01.txt')
ranged_biosignals = get_biosignal_in_range(biosignals, 0, 62)
tetris_data = read_tetris_data('s01/tetris_first_round.txt')
first_player_tetris_result = get_first_player_tetris_state(tetris_data)


#merge_biosignals_with_tetris_results(range_biosignals, first_player_tetris_result)

insert_tetris_result(ranged_biosignals, first_player_tetris_result)
ranged_biosignals = pd.get_dummies(ranged_biosignals, prefix='', prefix_sep='')

/Users/krzysztofwojdak/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/krzysztofwojdak/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
